In [1]:
import os
import sys
import git
import pathlib

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

print(f"Project Root Directory: {PROJ_ROOT}")

Project Root Directory: /repos/drl_csense


In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import scipy.fftpack as spfft
from sklearn.linear_model import Lasso

In [4]:
import imageio
import ipyplot

In [5]:
def idct2(x):
        return spfft.idct(spfft.idct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

alpha=1E-5
max_iter=1E3


In [6]:
def reconstruct_frame(frame, lasso):
    ny, nx = frame.shape
    A = np.kron(
        spfft.idct(np.identity(nx), norm='ortho', axis=0),
        spfft.idct(np.identity(ny), norm='ortho', axis=0)
        )

    flat_frame = frame.T.reshape(-1,1)
    idx_nonzero = np.where(flat_frame != 0)[0]

    # compressed sample
    b = flat_frame[idx_nonzero]
    # compression matrix @ transform matrix
    Ac = A[idx_nonzero,:]

    # LASSO optimization
    lasso.fit(Ac, b)

    # Reconstruct frame
    Xat = np.array(lasso.coef_).reshape(nx, ny).T # stack columns

    # Get the reconstructed frame
    Xa = idct2(Xat)

    return Xa

In [7]:
# env_id = "BreakoutNoFrameskip-v4"
# noise = 0.4

ENV_LIST = ["BreakoutNoFrameskip-v4", "BankHeistNoFrameskip-v4", "WizardOfWorNoFrameskip-v4"]
NOISE_LIST = [0.0, 0.1, 0.2, 0.3, 0.4]

In [8]:
for env_id in ENV_LIST:
    for noise in NOISE_LIST:
        lasso = Lasso(alpha=alpha, max_iter=int(max_iter), warm_start=True, selection="random")
        print("--"*10)
        print(f"{env_id=}; {noise=}")    
        obs_gif_file = f"./gifs/{env_id}--noise_{noise}--obs.gif"
        obs_gif_frames = imageio.mimread(obs_gif_file)
        
        reconstructed_frame_list = []
        for i,gif_frame in enumerate(obs_gif_frames[:50]):
            print(f"Processing Frame: {i}", end='\x1b[1K\r')
            # Take only one of the layers and normalize
            frame = gif_frame[:,:,0]/255
            reconstructed_frame = reconstruct_frame(frame, lasso)
            # Denormalize
            reconstructed_frame = 255 * (reconstructed_frame - np.min(reconstructed_frame))/(np.max(reconstructed_frame) - np.min(reconstructed_frame))
            reconstructed_frame_list.append(reconstructed_frame)
        # Convert obss to animation
        reconstructed_gif_file = f"./gifs/reconstructed/{env_id}--noise_{noise}--obs--reconstructed_v2.gif"
        imageio.mimsave(reconstructed_gif_file, 
                        [np.array(reconstructed_frame) for i, reconstructed_frame in enumerate(reconstructed_frame_list)], duration=100)

--------------------
env_id='BreakoutNoFrameskip-v4'; noise=0.0


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.714e-02, tolerance: 6.362e-03
  model = cd_fast.enet_coordinate_descent(


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.005e-02, tolerance: 6.411e-03
  model = cd_fast.enet_coordinate_descent(


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.511e-02, tolerance: 6.408e-03
  model = cd_fast.enet_coordinate_descent(


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.854e-02, tolerance: 6.499e-03
  model = cd_fast.enet_coordinate_descent(


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.737e-02, tolerance: 5.678e-03
  model = cd_fast.enet_coordinate_descent(


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.429e-02, tolerance: 5.626e-03
  model = cd_fast.enet_coordinate_descent(


--------------------
env_id='BreakoutNoFrameskip-v4'; noise=0.3


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.966e-02, tolerance: 4.997e-03
  model = cd_fast.enet_coordinate_descent(


--------------------
env_id='BreakoutNoFrameskip-v4'; noise=0.4
--------------------
env_id='BankHeistNoFrameskip-v4'; noise=0.0
--------------------
env_id='BankHeistNoFrameskip-v4'; noise=0.1
--------------------
env_id='BankHeistNoFrameskip-v4'; noise=0.2
--------------------
env_id='BankHeistNoFrameskip-v4'; noise=0.3
--------------------
env_id='BankHeistNoFrameskip-v4'; noise=0.4
--------------------
env_id='WizardOfWorNoFrameskip-v4'; noise=0.0
--------------------
env_id='WizardOfWorNoFrameskip-v4'; noise=0.1
--------------------
env_id='WizardOfWorNoFrameskip-v4'; noise=0.2
--------------------
env_id='WizardOfWorNoFrameskip-v4'; noise=0.3
--------------------
env_id='WizardOfWorNoFrameskip-v4'; noise=0.4


In [9]:
# for eval_param_value in reversed([0.0,0.1,0.2, 0.3, 0.4]):
#     alpha = 1E-5
#     print()
#     print(f"{eval_param_value=}")
#     # Get names and tags of experiment
#     exp_name, exp_metaname, exp_tag = get_exp_name_tag(env_id, exp_param_type, exp_param_value)
    
#     # Get directories
#     models_dir, log_dir, gif_dir, image_dir = makeget_logging_dir(exp_name)
    
#     # Load gif file
#     obs_gif_file = f"{gif_dir}/{exp_name}-run_{run_no}--eval_{model_type}-{eval_param_type}_{eval_param_value}--obs.gif"
    
#     obs_gif_frames = imageio.mimread(obs_gif_file)
    
#     reconstructed_frame_list = []
#     for i,gif_frame in enumerate(obs_gif_frames[:50]):
#         print(f"Processing Frame: {i}", end='\x1b[1K\r')
#         # Take only one of the layers and normalize
#         frame = gif_frame[:,:,0]/255
#         reconstructed_frame = reconstruct_frame(frame, alpha=alpha, max_iter=1E4)
#         reconstructed_frame = 255 * (reconstructed_frame - np.min(reconstructed_frame))/(np.max(reconstructed_frame) - np.min(reconstructed_frame))
#         reconstructed_frame_list.append(reconstructed_frame)
#         # reconstructed_frame_list.append(frame*255)
        
    
#     # Convert obss to animation
#     reconstructed_gif_file = f"{eval_param_type}_{eval_param_value}--reconstructed-alpha_1e-5.gif"
#     imageio.mimsave(reconstructed_gif_file, 
#                     [np.array(reconstructed_frame) for i, reconstructed_frame in enumerate(reconstructed_frame_list)], duration=100)

In [10]:
# ENV_LIST = ["BreakoutNoFrameskip-v4", "BankHeistNoFrameskip-v4", "WizardOfWorNoFrameskip-v4"]



# # env_id = "BreakoutNoFrameskip-v4"
# # noise = 0.4
# NOISE_LIST = [0.0, 0.1, 0.2, 0.3, 0.4]
# for env_id in ENV_LIST:
#     for noise in NOISE_LIST:
#         wrapper = AtariWrapper_NoisyWarp
#         wrapper_kwargs = {"noise":noise}
        
#         trial_env = make_atari_env_Custom_VecFrameStack(env_id=env_id,
#                                                         n_envs=1,
#                                                         monitor_dir=None,
#                                                         seed=161803,
#                                                         wrapper_class=wrapper,
#                                                         wrapper_kwargs=wrapper_kwargs
#                                                          )
        
        
        
#         # Generate frames
#         duration = 5
#         obss, images = generate_random_frames(trial_env, duration=duration, fps=200) # generates fps*duration*2 frames
            
        
#         # Convert frames to animation
#         img_gif_file = f"{env_id}--noise_{noise}--img.gif"
#         imageio.mimsave(img_gif_file, 
#                         [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=duration)
        
#         # Convert obss to animation
#         obs_gif_file = f"{env_id}--noise_{noise}--obs.gif"
#         imageio.mimsave(obs_gif_file, 
#                         [np.array(obs) for i, obs in enumerate(obss) if i%2 == 0], duration=duration)
        
#         trial_env.close()